# Practicals for lecture 1.2

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vigji/python-cimec-2025/blob/main/practicals/Practicals_1.2.ipynb)

#### 1.2.0 

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
def generate_RT_data(n_subjects=200, n_samples_per_subject=1000):
    """Generates Reaction Time data for a given number of subjects"""
    np.random.seed(0)  # For reproducibility
    shift = 0.500  # Shift of the distribution
    # Initialize an empty array to store the RT times for all subjects
    RT_data = np.empty((n_subjects, n_samples_per_subject))
    
    for i in range(n_subjects):
        # Assuming mu ranges from 90 to 110 and sigma from 10 to 20 for the subjects
        mu = np.random.uniform(0.090, 0.110)
        sigma = np.random.uniform(0.10, 0.20)
        RT_data[i] = np.random.normal(mu, sigma, n_samples_per_subject) + shift
    
    return RT_data

# This is our data matrix (200 subjects, 1000 RTs each)
rt_data = generate_RT_data()

# Assume that w consider outliers the RTs longer than 0.7 seconds. 
# Compute again the mean RTs after having excluded such outliers (make sure you use only vector operations)

# (Hint: an easy way to exclude outliers without using loops is by setting nans in the matrix)


In [ ]:
# Find the shortest RT of the whole dataset; then, find its index to know the subject and the trial number:


In [ ]:
def download_meteo_data(start_date="2022-01-01", end_date="2022-12-31",
                        latitude="45.88204", longitude="11.03647",
                        data="temperature_2m"):
    """Download meteo historical data from open-meteo.com."""

    import requests
    import json

    BASE_URL = "https://archive-api.open-meteo.com/v1/"
    query = f"archive?latitude={latitude}&longitude={longitude}&start_date={start_date}&end_date={end_date}&hourly={data}"

    r = requests.get(BASE_URL + query)
    json_dict = json.loads(r.text)
    
    if "hourly" not in json_dict.keys():
        print(json_dict)
        return None, None
    else:
        return np.array(json_dict["hourly"]["time"]).reshape(-1, 24), np.array(json_dict["hourly"][data]).reshape(-1, 24)

# Find the index for the highest temperature in the temperature_array.
# Then, apply the index you have found over timestamps_array to read out the corresponding timestamp(s).

# (Hint:
# Remember, you have to either: 
#  - use np.argmax, and to work with the flattened() arrays when using the index you found;
#  - find maximum value using np.max, and then use np.nonzero to get the indexes of values 
#    equal to the max over both dimensions of the array)

timestamps_array, temperatures_array = download_meteo_data()

In [4]:
# The following code gives you a matrix of zeros and ones:
import numpy as np
np.random.seed(42)
data = np.random.randint(0, 2, size=(5, 6))


# Find the rows and columns of all the ones in the matrix:

In [ ]:
# Then, replace them with nans using the indexes you found; 
# can you do it without changing the dtype of the array?

In [ ]:
# This array represents a stupid example signal with some noise.
# Without using loops, find the indexes of the peaks in the signal.
# Peaks are defined as consecutive numbers in the array that are larger than the numbers immediately before and after them.

signal = np.array([1, 3, 7, 1, 2, 6, 0, 1])

#### Practicals 1.2.1

In [ ]:
# Now compute the 1D array of average temperatures per day.
# Build a boolean selector to filter all mean temperatures above 10 and below 25 degrees:


In [44]:
# For the array of integer numbers below, use array boolean operations to filter out the numbers that 
# are greater than 5 AND less than 8, OR that are multiple of 7.
# USe the selector to create a new array with the included numbers.
np.random.seed(42)
an_array = np.random.randint(0, 10, 100)


In [ ]:
# We can use the np.argsort() function to produce the indexes array required to
# order an array in ascending or descending values.

# For example:
random_arr = np.array([0.1, 5, 3.4, 2.3])
ordering_idxs = np.argsort(random_arr)
random_arr[ordering_idxs]  # with this index, this is now ordered!

# Let's make a ranking of the 5 warmest hours of the dataset!
# Sort the (flattened) temperature array using the indexes produced by np.argsort.
# so that the first elements are the highest temperatures.
# Then sort the timestamps array with the same indexes, and take the first 5.
#
# Double check you match the result that you have got in the exercises above!

#### 1.2.2 Introduction to `pandas` DataFrames

In [45]:
import pandas as pd

In [ ]:
# Consider the following dataset with info about subject in an experiment:
np.random.seed(42)
n_subjects = 100
subjects_df = pd.DataFrame({
    'age': np.random.randint(20, 40, n_subjects),
    'weight': np.random.randint(50, 100, n_subjects),
    'height': np.random.randint(150, 200, n_subjects),
    'sex': np.random.choice(['M', 'F'], n_subjects),
    'handedness': np.random.choice(['R', 'L'], n_subjects),
    'group': np.random.choice(['control', 'patient'], n_subjects)})

subjects_df.head()

In [ ]:
# Select the first two rows of the dataframe:


In [ ]:
# Select the column of the dataframe containing the subjects weight using the name of the column:


In [ ]:
# Select the `weight` column of the dataframe filtering only rows of subjects > 34 years old:


In [ ]:
# Count how many males and how many females above age 30 are left-handed or right handed.
# (Hint: use the same element-wise operators we were using for numpy arrays)


In [ ]:
# Use .iloc to select one every two rows for the first 3 columns:


In [ ]:
# Redefine the dataset index to be "subject_n_sex" 
# (where n is progressive number of the subject and sex the sex of the subject)


#### 1.2.2

In [ ]:
# Create a dataframe from the following info about subject in an experiment:
np.random.seed(42)
n_subjects = 100
subjects_data_dictionary = {
    'age': np.random.randint(20, 40, n_subjects),
    'weight': np.random.randint(50, 100, n_subjects),
    'height': np.random.randint(150, 200, n_subjects),
    'sex': np.random.choice(['M', 'F'], n_subjects),
    'handedness': np.random.choice(['R', 'L'], n_subjects),
    'group': np.random.choice(['control', 'patient'], n_subjects)}


,age,weight,height,sex,handedness,group
0,26,84,175,M,L,patient
1,39,93,181,M,R,patient
2,34,89,155,F,L,patient
3,30,71,181,M,R,patient
4,27,76,153,F,L,control


In [ ]:
# We can calculate the BMI as :
# (body weight in kgs) /  ( (height in meters) ** 2) 

# Define a new column where you compute the BMI for every subject:


,age,weight,height,sex,handedness,group,height_m,BMI
0,26,84,175,M,L,patient,1.75,27.428571
1,39,93,181,M,R,patient,1.81,28.387412
2,34,89,155,F,L,patient,1.55,37.044745
3,30,71,181,M,R,patient,1.81,21.672110
4,27,76,153,F,L,control,1.53,32.466145


In [ ]:
string_data = ['cabbage', 'artichoke', 'banana', 'avocado', 'apple', 'orange']
int_data = [1, 2, 3, 4, 5, 6]
float_data = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]

# organize the data above in a dictionary and from it define a new DataFrame:


,string_data,int_data,float_data
0,cabbage,1,0.1
1,artichoke,2,0.2
2,banana,3,0.3
3,avocado,4,0.4
4,apple,5,0.5


In [ ]:
# Now, reorganize the data in a list of dictionaries 
# (one for every row) and from it define a new DataFrame:



,string_data,int_data,float_data
0,cabbage,1,0.1
1,artichoke,2,0.2
2,banana,3,0.3
3,avocado,4,0.4
4,apple,5,0.5
5,orange,6,0.6


In [ ]:
# Extend the subject dataset above adding two subjects for which you have the following info:
# (Hint: start by defining a new dataframe, and then concatenate it)

new_subject1 = {'age': 25,
    'weight': 78,
    'height': 170,
    'sex': "M",
    'handedness': "R",
    'group': "patient"}
new_subject2 = {'age': 35,
    'weight': 65,
    'height': 165,
    'sex': "F",
    'handedness': "R",
    'group': "control"}

# Make sure the new dataset has univoque indexing! 


,age,weight,height,sex,handedness,group
0,25,78,170,M,R,patient


In [ ]:
# [Advanced]
# In the above exercise, we're 'adding new rows" to our dataframe with pd.concat.
# Is this process truly modifying the original subjects_df object or are we creating a new object in memory? Write down your guess then verify it using python's "is" keyword.
# (as in "df_a is df_b"; will return True if df_a and df_b are the same object in memory)

# Based on this information, would it be efficient to add a large number of subjects to a dataframe in a for loop using pd.concat?


In [ ]:
# Iteratively growing a dataframe
# write 4 different functions for iteratively adding subject numbers (from 1 to 100) row by row to an empty dataframe
# to be clear, the final output is a dataframe with column name "subject" and 100 rows [0,1,2,...100] - very simple output but we want to know which approach is most efficient


def append_with_concat():
  df = pd.DataFrame(columns = ['subject']) # create an empty dataframe with only the column name
  for subject_num in range(100):
     ...
    #create a dataframe to store subject_num (i.e. 1 row), and add it to df using pd.concat"

def collect_dfs_then_concat():
  list_of_dfs = []
  for subject_num in range(100):
     ...
    #create a dataframe to store subject_num (i.e. 1 row) then add this dataframe to the list_of_dfs
  #outside the for loop, combine all the dfs from the list_of_dfs as well as the empty one into a single df with pd.concat

def append_to_list_then_convert_to_df():
  list_of_subject_nums = []
  for subject_num in range(100):
     ...
    #append to the list
  #outside the for loop, convert the list to a df

def preallocate_numpy_array():
    array = np.zeros((100, 1))
    for subject_num in range(100):
       ...
        #replace the zeros in the array with the subject_num
    #outside the for loop, convert the array into a df

In [ ]:
# let's compare how fast each option is!
%timeit append_with_concat()
%timeit collect_dfs_then_concat()
%timeit append_to_list_then_convert_to_df()
%timeit preallocate_numpy_array()
# the example was ridiculously simple (100 rows and 1 column) so if you're working with a larger dataset you'll feel the difference between these even more
# in general, you should really avoid appending to a dataframe. If you need to iteratively construct a data container, work with lists, arrays or dicts then only convert to
# dataframes at the end for extracting info from your final dataframe